# Artificial Intelligence Project 2025/26  
## Class Timetable (CSP Problem)

---

### 1. Introduction
> This project aims to develop an intelligent agent capable of generating class timetables for the undergraduate courses at the Polytechnic Institute of Cávado and Ave (IPCA).  
> Creating timetables is a complex and time-consuming task that involves multiple constraints related to professors, subjects, classrooms, and student groups. Traditionally, this process requires significant time and effort from the administrative team.  
> In this context, the project intends to apply Artificial Intelligence techniques, specifically **Constraint Satisfaction Problems (CSP)**, to automate the creation of viable and optimized timetables, respecting the imposed conditions and minimizing conflicts.  
> 
> The project is implemented in **Python**, using the **Google OR-Tools CP-SAT solver** (`ortools.sat.python.cp_model`), which provides a powerful and efficient framework for modeling and solving CSPs and combinatorial optimization problems.  
> The entire process is documented in a Jupyter Notebook, as per the guidelines of the Artificial Intelligence (AI) course.

**Group members:**
- Pedro Ribeiro — student number 27960  
- Ricardo Fernandes — student number 27961  
- Carolina Branco — student number 27983  
- João Barbosa — student number 27964  
- Diogo Abreu — student number 27975  

---

### 2. Goal Formulation

The main goal of this project is to **design an intelligent agent** capable of automatically generating valid class timetables for undergraduate courses at IPCA.  

The agent must assign **courses to time slots, rooms, and teachers**, ensuring that no scheduling conflicts occur and that all academic and logistical constraints are satisfied.  

#### Limitations and Constraints
The timetable generation process is subject to several limitations, including:

- **Teacher availability** — each lecturer may have unavailable time slots.  
- **Room capacity and restrictions** — some courses must occur in specific rooms (e.g., labs).  
- **Course assignment** — each class is associated with a specific set of courses.  
- **Online lessons** — some sessions may occur online and not require a physical room.  
- **Non-overlapping sessions** — a class, teacher, or room cannot be used in two sessions simultaneously.  
- **Class duration** — each lesson lasts 2 hours.  
- **Weekly lessons per class** — all classes have 10 lessons per week.  
- **Lessons per course** — each course may have 1 or 2 lessons per week.  
- **Daily lesson limit** — a class cannot have more than 3 lessons per day.  

#### Expected Results
The system should produce a **valid timetable solution** that:
- Assigns every course session to a valid time slot and room.  
- Respects all **hard constraints**.  
- Minimizes conflicts and overlaps between classes, teachers, and rooms.  
- Optionally considers **soft constraints** to improve the quality of the timetable (e.g., minimizing room changes or distributing lessons across the week).  

---

### 3. Problem Formulation (CSP)

The timetable problem was modeled as a **Constraint Satisfaction Problem (CSP)** and solved using the **Google OR-Tools CP-SAT solver** (`ortools.sat.python.cp_model`).  
This solver supports **integer variables**, **Boolean logic**, **complex constraints**, and **custom decision strategies**, making it suitable for this type of scheduling problem.

#### Variables
Each **lesson** of a course is represented by a group of integer variables:
- `<course>_<lesson>_slot` — the time slot assigned to that lesson (`IntVar` in range 1–20).  
- `<course>_<lesson>_room` — the classroom assigned to that lesson (`IntVar` representing room indices).  
- `<course>_<lesson>_teacher` — the lecturer assigned to that lesson.  
- `<course>_<lesson>_day` — the day of the week (0–4).  

#### Domains
- **Time slots:** integers `1–20` (representing sequential periods across Monday–Friday).  
- **Rooms:** `{Lab01, Room1, Room2, Online}`, encoded as integer indices.  
- **Days:** automatically derived from slots (`(slot − 1) // 4`).  
- **Teachers:** integer indices mapped from names.

#### Constraints

**Hard Constraints (mandatory):**
- Lessons last 2 hours each.  
- All classes have 10 lessons per week.  
- Each course has 1 or 2 lessons per week.  
- Each class can have at most **3 lessons per day**.  
- Teachers cannot teach in **unavailable slots** (from dataset).  
- **Room restrictions:** certain courses must be assigned to specific rooms (e.g., labs).  
- **No overlaps:**  
  - Two lessons cannot share the same room and slot.  
  - A teacher or class cannot be assigned to two lessons simultaneously.  
- **Online courses:** forced to occur in the “Online” room.  
- **Non-consecutive lessons:** lessons of the same course are not allowed in consecutive slots.

**Soft Constraints (preferred):**
- Lessons of the same course should be scheduled on different days.  
- Classes should ideally use as few different rooms as possible (minimize room changes for teachers and students).  
- Classes should have four active days per week if possible.  

#### Additional Constraints (implemented or optional):
- A class may not exceed three lessons per day.  
- Online lessons for the same course should occur on the same day when possible.  

---

### 4. Heuristics

To improve search efficiency, **custom decision strategies** (heuristics) were applied to guide the solver:  

- **Variable ordering heuristic:**  
  The solver uses the **Minimum Remaining Values (MRV)** strategy (`CHOOSE_MIN_DOMAIN_SIZE`), selecting first the variable with the fewest remaining valid options.  

- **Value ordering heuristic:**  
  The **Least Constraining Value (LCV)** strategy (`SELECT_MIN_VALUE`) is used, prioritizing assignments that restrict fewer future choices.  

These heuristics significantly reduce the search space and improve runtime performance.  
They are implemented via:
```python
model.AddDecisionStrategy(
    slot_var_list,
    cp_model.CHOOSE_MIN_DOMAIN_SIZE,
    cp_model.SELECT_MIN_VALUE
)
```
and optionally extended with external helpers for configurable heuristics:

```python
from src.helpers import escolher_heuristica_variaveis, escolher_heuristica_valores
```

### 5. Solution Evaluation and Optimization

During the solving process, all feasible solutions are explored within a maximum time limit (`MAX_TIME_SECONDS = 30`).
A custom Solution Collector class (`CpSolverSolutionCallback`) stores each valid solution, evaluates them, and selects the best one according to a soft optimization criterion:

> Goal: minimize the number of room changes per teacher across their weekly schedule.

This scoring heuristic ensures that teachers’ schedules are more stable and less fragmented.
The best-scoring timetable is then exported to a JSON file for further analysis and visualization.

### 6. Implementation Summary

The implementation pipeline can be summarized as follows:

**1. Data Loading:**

Read dataset from `datasets/timetable_dataset.txt` and extract:

- Courses assigned to classes

- Courses assigned to lecturers

- Time slot restrictions

- Room restrictions

- Online class mappings

**2. Model Definition:**

- Create integer variables for slots, rooms, teachers, and days.

- Apply domain and constraint definitions programmatically.

**3. Constraint Enforcement:**

- Add logical, arithmetic, and “AllDifferent” constraints using OR-Tools API.

- Guarantee class, teacher, and room consistency.

**4. Heuristic Configuration:**

- Apply MRV and LCV heuristics to improve efficiency.

**5. Solving and Optimization:**

- Execute CP-SAT search with time limit and optional full enumeration.

- Collect and score feasible solutions.

**6. Output Generation:**

- Export the best timetable to `outputs/best_schedule.json`.

- Print summary of variable assignments and performance statistics.

### 7. Expected Results

The final system automatically produces valid, conflict-free class timetables that:

- Respect all hard constraints;

- Consider teachers’ unavailability;

- Handle both in-person and online classes;

- Optimize soft criteria (minimize room changes);

- Provide results within seconds.

This demonstrates the application of Artificial Intelligence — specifically Constraint Programming — to a real-world scheduling problem, effectively reducing manual workload and improving timetable quality.

### 8. Repository Link
> 🔗 **GitHub Repository:** [https://github.com/diogooaabreu/IA25_P01_G4.git](https://github.com/diogooaabreu/IA25_P01_G4.git)

---






In [ ]:
import json
import time
from collections import Counter
from ortools.sat.python import cp_model

# ======================================================
# 1. BASIC CONFIGURATION AND CONSTANTS
# ======================================================

rooms = ["Lab01", "Room1", "Room2", "Online"]
slots = list(range(1, 21))  
room_to_int = {r: i for i, r in enumerate(rooms)}
int_to_room = {i: r for r, i in room_to_int.items()}

DEFAULT_LESSONS_PER_COURSE = 2

# ======================================================
# 2. HELPER FUNCTIONS
# ======================================================

def build_teacher_mappings(courses_to_lecturers):
    """Create integer mappings for teacher identifiers."""
    teachers = sorted(courses_to_lecturers.keys())
    t2i = {t: i for i, t in enumerate(teachers)}
    i2t = {i: t for t, i in t2i.items()}
    return t2i, i2t

def get_day_from_slot(slot):
    """Return the day index corresponding to a slot number.

    Slots are numbered sequentially, com 4 slots por dia.
    Monday = 0, Tuesday = 1, ..., Friday = 4.
    """
    return (slot - 1) // 4

# ======================================================
# 3. DATA LOADING 
# ======================================================

dataset_path = "datasets/timetable_dataset.txt"

courses_assigned_to_classes = {}
courses_assigned_to_lecturers = {}
timeslot_restrictions = {}
roomrestrictions = {}
online_classes = {}
all_courses_with_variables = set()

try:
    with open(dataset_path, "r", encoding="utf-8-sig") as f:
        lines = f.readlines()

    reading_section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        elif line.startswith("#cc"):
            reading_section = "cc"
        elif line.startswith("#dsd"):
            reading_section = "dsd"
        elif line.startswith("#tr"):
            reading_section = "tr"
        elif line.startswith("#rr"):
            reading_section = "rr"
        elif line.startswith("#oc"):
            reading_section = "oc"
        elif line.startswith("#"):
            reading_section = None
        else:
            if reading_section == "cc":
                parts = line.split()
                courses_assigned_to_classes[parts[0]] = parts[1:]
                for course in parts[1:]:
                    all_courses_with_variables.add(course)
            elif reading_section == "dsd":
                parts = line.split()
                courses_assigned_to_lecturers[parts[0]] = parts[1:]
            elif reading_section == "tr":
                parts = line.split()
                timeslot_restrictions[parts[0]] = list(map(int, parts[1:]))
            elif reading_section == "rr":
                parts = line.split()
                roomrestrictions[parts[0]] = parts[1:]
            elif reading_section == "oc":
                parts = line.split()
                online_classes[parts[0]] = list(map(int, parts[1:]))

except FileNotFoundError:
    print(f"WARNING: File not found at path: {dataset_path}")
    print("Using simulated data to continue the example.")
    courses_assigned_to_classes = {
        't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15',
                'UC21', 'UC22', 'UC23', 'UC24', 'UC25',
                'UC31', 'UC32', 'UC33', 'UC34', 'UC35']
    }
    courses_assigned_to_lecturers = {
        'jo': ['UC11', 'UC21', 'UC22', 'UC31'],
        'mike': ['UC12', 'UC23', 'UC32'],
        'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
        'sue': ['UC15', 'UC25', 'UC34', 'UC35']
    }
    timeslot_restrictions = {
        'mike': [13, 14, 15, 16, 17, 18, 19, 20],
        'rob': [1, 2, 3, 4],
        'sue': [9, 10, 11, 12, 17, 18, 19, 20]
    }
    roomrestrictions = {'UC14': ['Lab01'], 'UC22': ['Lab01']}
    online_classes = {'UC21': [2], 'UC31': [2]}
    all_courses_with_variables = set(courses_assigned_to_classes['t01'])

num_lessons_by_course = {c: 2 for c in all_courses_with_variables}

# ======================================================
# 4. MODEL CREATION
# ======================================================

model = cp_model.CpModel()
teacher_to_int, int_to_teacher = build_teacher_mappings(courses_assigned_to_lecturers)

slot_vars = {}
room_vars = {}
teacher_vars = {}
day_vars = {}
is_online_vars = {}

# ======================================================
# 5. VARIABLE DEFINITIONS (agora por curso com L = 1 ou 2)
# ======================================================

for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
        for lesson in range(1, L + 1):
            key = (course, lesson)
            s = model.NewIntVar(min(slots), max(slots), f"{course}_{lesson}_slot")
            slot_vars[key] = s

            min_room_idx = min(room_to_int.values())
            max_room_idx = max(room_to_int.values())
            r = model.NewIntVar(min_room_idx, max_room_idx, f"{course}_{lesson}_room")
            if course in roomrestrictions:
                allowed_rooms = roomrestrictions[course]
                allowed_room_idxs = [room_to_int[rn] for rn in allowed_rooms if rn in room_to_int]
                model.AddAllowedAssignments([r], [[v] for v in allowed_room_idxs])
            room_vars[key] = r

            teachers_for_course = [t for t, c_list in courses_assigned_to_lecturers.items() if course in c_list]
            if teachers_for_course:
                teacher_idxs = [teacher_to_int[t] for t in teachers_for_course]
                tvar = model.NewIntVar(min(teacher_to_int.values()), max(teacher_to_int.values()), f"{course}_{lesson}_teacher")
                model.AddAllowedAssignments([tvar], [[v] for v in teacher_idxs])
                teacher_vars[key] = tvar

            dvar = model.NewIntVar(0, 4, f"{course}_{lesson}_day")
            day_vars[key] = dvar

            is_online = model.NewBoolVar(f"{course}_{lesson}_is_online")
            is_online_vars[key] = is_online
            online_idx = room_to_int["Online"]
            model.Add(r == online_idx).OnlyEnforceIf(is_online)
            model.Add(r != online_idx).OnlyEnforceIf(is_online.Not())

# ======================================================
# 6. CONSTRAINTS
# ======================================================

# Ligação slot <-> day (permitir apenas combinações válidas: slot -> day)
slot_day_allowed = [[s, get_day_from_slot(s)] for s in slots]
for key, s_var in slot_vars.items():
    d_var = day_vars[key]
    model.AddAllowedAssignments([s_var, d_var], slot_day_allowed)

# Impedir aulas consecutivas da mesma UC **apenas** quando na mesma dia
for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
        lesson_keys = [(course, lesson) for lesson in range(1, L + 1)]
        for i in range(len(lesson_keys)):
            for j in range(i + 1, len(lesson_keys)):
                k1, k2 = lesson_keys[i], lesson_keys[j]
                if k1 not in slot_vars or k2 not in slot_vars:
                    continue
                s1, s2 = slot_vars[k1], slot_vars[k2]
                d1, d2 = day_vars[k1], day_vars[k2]
                same_day = model.NewBoolVar(f"same_day_{k1}_{k2}")
                model.Add(d1 == d2).OnlyEnforceIf(same_day)
                model.Add(d1 != d2).OnlyEnforceIf(same_day.Not())
                model.Add(s1 - s2 != 1).OnlyEnforceIf(same_day)
                model.Add(s2 - s1 != 1).OnlyEnforceIf(same_day)

# Impedir conflito sala+slot (hard): se r1 == r2 então s1 != s2
all_course_list = list(all_courses_with_variables)
for i in range(len(all_course_list)):
    for j in range(i + 1, len(all_course_list)):
        c1, c2 = all_course_list[i], all_course_list[j]
        L1 = num_lessons_by_course.get(c1, DEFAULT_LESSONS_PER_COURSE)
        L2 = num_lessons_by_course.get(c2, DEFAULT_LESSONS_PER_COURSE)
        for l1 in range(1, L1 + 1):
            for l2 in range(1, L2 + 1):
                key1, key2 = (c1, l1), (c2, l2)
                if key1 in room_vars and key2 in room_vars and key1 in slot_vars and key2 in slot_vars:
                    r1, r2 = room_vars[key1], room_vars[key2]
                    s1, s2 = slot_vars[key1], slot_vars[key2]

                    same_room = model.NewBoolVar(f"same_room_{c1}_{l1}__{c2}_{l2}")
                    model.Add(r1 == r2).OnlyEnforceIf(same_room)
                    model.Add(r1 != r2).OnlyEnforceIf(same_room.Not())
                    model.Add(s1 != s2).OnlyEnforceIf(same_room)
                    model.Add(s1 - s2 != 1).OnlyEnforceIf(same_room)
                    model.Add(s2 - s1 != 1).OnlyEnforceIf(same_room)

# Disponibilidade de professores (timeslot_restrictions)
for teacher, courses in courses_assigned_to_lecturers.items():
    unavailable_slots = timeslot_restrictions.get(teacher, [])
    for course in courses:
        L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
        for lesson in range(1, L + 1):
            key = (course, lesson)
            if key in slot_vars and key in teacher_vars:
                tvar = teacher_vars[key]
                svar = slot_vars[key]
                for unavailable in unavailable_slots:
                    if teacher in teacher_to_int:
                        t_idx = teacher_to_int[teacher]
                        is_this_teacher = model.NewBoolVar(f"is_{teacher}_{course}_{lesson}")
                        model.Add(tvar == t_idx).OnlyEnforceIf(is_this_teacher)
                        model.Add(tvar != t_idx).OnlyEnforceIf(is_this_teacher.Not())
                        model.Add(svar != unavailable).OnlyEnforceIf(is_this_teacher)

# Garantir slots únicos por turma (não duas aulas da mesma turma no mesmo slot)
for class_name, courses in courses_assigned_to_classes.items():
    lesson_slots = []
    for course in courses:
        L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
        for lesson in range(1, L + 1):
            key = (course, lesson)
            if key in slot_vars:
                lesson_slots.append(slot_vars[key])
    if lesson_slots:
        model.AddAllDifferent(lesson_slots)

# Garantir slots únicos por professor (não podem lecionar duas aulas simultâneas)
for teacher, courses in courses_assigned_to_lecturers.items():
    lesson_slots = []
    for course in courses:
        L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
        for lesson in range(1, L + 1):
            key = (course, lesson)
            if key in slot_vars and key in teacher_vars:
                lesson_slots.append(slot_vars[key])
    if lesson_slots:
        model.AddAllDifferent(lesson_slots)

# Forçar online courses para sala "Online" quando indicado no ficheiro
for course, week_indexes in online_classes.items():
    if course in all_courses_with_variables:
        for lesson in week_indexes:
            key = (course, lesson)
            if key in room_vars and key in is_online_vars:
                model.Add(room_vars[key] == room_to_int["Online"])
                model.Add(is_online_vars[key] == 1)

# Limitar número de aulas por dia por turma (<= 3)
for class_name, courses in courses_assigned_to_classes.items():
    lesson_day_vars = []
    for course in courses:
        L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
        for lesson in range(1, L + 1):
            key = (course, lesson)
            if key in day_vars:
                lesson_day_vars.append(day_vars[key])

    if lesson_day_vars:
        for d in range(5):
            indicators = []
            for idx, dvar in enumerate(lesson_day_vars):
                b = model.NewBoolVar(f"{class_name}_day{d}_lesson{idx}")
                model.Add(dvar == d).OnlyEnforceIf(b)
                model.Add(dvar != d).OnlyEnforceIf(b.Not())
                indicators.append(b)
            model.Add(sum(indicators) <= 3)

# Cada turma tem no máximo 10 lições por semana (hard constraint do enunciado original)
for class_name, courses in courses_assigned_to_classes.items():
    total_lessons = sum(num_lessons_by_course.get(c, DEFAULT_LESSONS_PER_COURSE) for c in courses)
    model.Add(total_lessons <= 10)

# Máximo de 3 aulas online por turma (hard)
online_idx = room_to_int["Online"]
for class_name, courses in courses_assigned_to_classes.items():
    lesson_keys = []
    for course in courses:
        L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
        for lesson in range(1, L + 1):
            lesson_keys.append((course, lesson))
    online_indicators = [is_online_vars[k] for k in lesson_keys if k in is_online_vars]
    if online_indicators:
        model.Add(sum(online_indicators) <= 3)
        for i in range(len(lesson_keys)):
            k_i = lesson_keys[i]
            if k_i not in is_online_vars or k_i not in day_vars:
                continue
            for j in range(i + 1, len(lesson_keys)):
                k_j = lesson_keys[j]
                if k_j not in is_online_vars or k_j not in day_vars:
                    continue
                model.Add(day_vars[k_i] == day_vars[k_j]).OnlyEnforceIf([is_online_vars[k_i], is_online_vars[k_j]])

# ======================================================
# 7. DECISION STRATEGIES (HEURISTICS)
# ======================================================

slot_var_list = list(slot_vars.values())
if slot_var_list:
    model.AddDecisionStrategy(
        slot_var_list,
        cp_model.CHOOSE_MIN_DOMAIN_SIZE,
        cp_model.SELECT_MIN_VALUE
    )

try:
    from src.helpers import escolher_heuristica_variaveis, escolher_heuristica_valores
    heuristica_vars_nome = "mrv"
    heuristica_vals_nome = "lcv"
    var_strategy = escolher_heuristica_variaveis(heuristica_vars_nome)
    val_strategy = escolher_heuristica_valores(heuristica_vals_nome)
    model.AddDecisionStrategy(
        [*slot_vars.values(), *room_vars.values(), *teacher_vars.values()],
        var_strategy,
        val_strategy
    )
except Exception:
    pass

# ======================================================
# 8. SOLVER CONFIGURATION AND EXECUTION
# ======================================================

solver = cp_model.CpSolver()
MAX_TIME_SECONDS = 30.0
solver.parameters.max_time_in_seconds = MAX_TIME_SECONDS
solver.parameters.enumerate_all_solutions = True

# ======================================================
# 9. SOLUTION COLLECTOR CLASS (corrigida)
# ======================================================

class SolutionCollector(cp_model.CpSolverSolutionCallback):
    """Collector that scores solutions according to soft constraints per class."""
    def __init__(self, slot_vars, room_vars, teacher_vars, day_vars, is_online_vars):
        super().__init__()
        self.slot_vars = slot_vars
        self.room_vars = room_vars
        self.teacher_vars = teacher_vars
        self.day_vars = day_vars
        self.is_online_vars = is_online_vars
        self.solution_count = 0
        self.best_solution = None
        self.best_score = None

    def OnSolutionCallback(self):
        self.solution_count += 1

        # recolher valores sem sobrescrever nomes globais
        slots_val = {k: self.Value(v) for k, v in self.slot_vars.items()}
        rooms_val = {k: self.Value(v) for k, v in self.room_vars.items()}
        teachers_val = {k: self.Value(v) for k, v in self.teacher_vars.items()}
        days_val = {k: self.Value(v) for k, v in self.day_vars.items()}
        is_online_val = {k: self.Value(v) for k, v in self.is_online_vars.items()}

        total_score = 0

        # Pesos de penalidade (menor = melhor)
        PEN_SAME_UC_SAME_DAY = 8
        PEN_EXTRA_ACTIVE_DAY = 3
        PEN_GAP_IN_DAY = 2
        PEN_ONLINE_DIFF_DAY = 10
        PEN_ONLINE_EXCEED = 15

        for class_name, courses in courses_assigned_to_classes.items():
            lesson_keys = []
            for course in courses:
                L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
                for lesson in range(1, L + 1):
                    key = (course, lesson)
                    if key in slots_val and key in days_val:
                        lesson_keys.append(key)

            # 1) Penalizar lições da mesma UC no mesmo dia
            for course in courses:
                L = num_lessons_by_course.get(course, DEFAULT_LESSONS_PER_COURSE)
                keys_uc = [(course, l) for l in range(1, L + 1) if (course, l) in days_val]
                if len(keys_uc) > 1:
                    seen_days = [days_val[k] for k in keys_uc]
                    counts = Counter(seen_days)
                    for day_val, cnt in counts.items():
                        if cnt > 1:
                            pairs = cnt * (cnt - 1) // 2
                            total_score += PEN_SAME_UC_SAME_DAY * pairs

            # 2) Penalizar número de dias activos (>4)
            days_active = set()
            for k in lesson_keys:
                days_active.add(days_val[k])
            num_active = len(days_active)
            if num_active > 4:
                total_score += PEN_EXTRA_ACTIVE_DAY * (num_active - 4)

            # 3) Penalizar gaps dentro de um dia
            for d in range(5):
                slot_list = [slots_val[k] for k in lesson_keys if days_val[k] == d]
                if len(slot_list) <= 1:
                    continue
                slot_list_sorted = sorted(slot_list)
                gap_count = (slot_list_sorted[-1] - slot_list_sorted[0] + 1) - len(slot_list_sorted)
                if gap_count > 0:
                    total_score += PEN_GAP_IN_DAY * gap_count

            # 4) Penalizar online >3 e online em dias diferentes
            online_keys = [k for k in lesson_keys if k in is_online_val and is_online_val[k] == 1]
            if len(online_keys) > 3:
                total_score += PEN_ONLINE_EXCEED * (len(online_keys) - 3)
            if len(online_keys) >= 2:
                online_days = {days_val[k] for k in online_keys}
                if len(online_days) > 1:
                    total_score += PEN_ONLINE_DIFF_DAY * (len(online_days) - 1)

        # Guardar melhor solução
        if self.best_score is None or total_score < self.best_score:
            self.best_score = total_score
            self.best_solution = {
                "slots": slots_val,
                "rooms": rooms_val,
                "teachers": teachers_val,
                "days": days_val,
                "is_online": is_online_val
            }

# ======================================================
# 10. OUTPUT AND RESULTS
# ======================================================

print("\n--- STARTING SEARCH (OR-Tools CP-SAT) ---")
print(f"Timeout: {MAX_TIME_SECONDS}s\n")

collector = SolutionCollector(slot_vars, room_vars, teacher_vars, day_vars, is_online_vars)
start = time.time()
status = solver.SearchForAllSolutions(model, collector)
elapsed = time.time() - start

print("Execution time:", elapsed, "seconds")
print("Status:", solver.StatusName(status))
print("Total solutions found:", collector.solution_count)
print("Best solution score (menor = melhor):", collector.best_score)

output_filepath = r"outputs\best_schedule.json"

if collector.solution_count > 0 and collector.best_solution is not None:
    print(f"\nSolution found. Exporting best to {output_filepath}")
    schedule = {}
    best = collector.best_solution

    for (course, lesson), s_val in best["slots"].items():
        lesson_key = f"{course}_{lesson}"
        schedule.setdefault(lesson_key, {})["slot"] = int(s_val)
        schedule.setdefault(lesson_key, {})["day"] = int(best["days"][(course, lesson)])

    for (course, lesson), r_val in best["rooms"].items():
        lesson_key = f"{course}_{lesson}"
        room_idx = int(r_val)
        schedule.setdefault(lesson_key, {})["room"] = int_to_room.get(room_idx, f"room_{room_idx}")

    for (course, lesson), t_val in best["teachers"].items():
        lesson_key = f"{course}_{lesson}"
        teacher_idx = int(t_val)
        schedule.setdefault(lesson_key, {})["teacher"] = int_to_teacher.get(teacher_idx, f"teacher_{teacher_idx}")

    sorted_schedule = dict(sorted(schedule.items(), key=lambda x: x[0]))

    import os
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    with open(output_filepath, "w", encoding="utf-8") as f:
        json.dump(sorted_schedule, f, indent=4)

    print("Successfully exported to JSON (ordered by UC).")
    print("\n--- Example of Assignments (first 50) ---")
    all_vars_print = []
    for (course, lesson), s_val in best["slots"].items():
        all_vars_print.append((f"{course}_{lesson}_slot", s_val))
    for (course, lesson), r_val in best["rooms"].items():
        room_name = int_to_room.get(int(r_val), f"room_{int(r_val)}")
        all_vars_print.append((f"{course}_{lesson}_room", room_name))
    for (course, lesson), t_val in best["teachers"].items():
        teacher_name = int_to_teacher.get(int(t_val), f"teacher_{int(t_val)}")
        all_vars_print.append((f"{course}_{lesson}_teacher", teacher_name))
    all_vars_print = sorted(all_vars_print, key=lambda x: x[0])
    for i, (name, val) in enumerate(all_vars_print):
        if i < 50:
            print(f"{name} → {val}")
else:
    print("No solution found.")




--- STARTING SEARCH (OR-Tools CP-SAT) ---
Timeout: 30.0s

Execution time: 30.005103588104248 seconds
Status: FEASIBLE
Total solutions found: 13887
Best solution score (menor = melhor): 14

Solution found. Exporting best to outputs\best_schedule.json
Successfully exported to JSON (ordered by UC).

--- Example of Assignments (first 50) ---
UC11_1_room → Room2
UC11_1_slot → 3
UC11_1_teacher → jo
UC11_2_room → Room2
UC11_2_slot → 19
UC11_2_teacher → jo
UC12_1_room → Online
UC12_1_slot → 11
UC12_1_teacher → mike
UC12_2_room → Room2
UC12_2_slot → 1
UC12_2_teacher → mike
UC13_1_room → Lab01
UC13_1_slot → 20
UC13_1_teacher → rob
UC13_2_room → Room2
UC13_2_slot → 12
UC13_2_teacher → rob
UC14_1_room → Lab01
UC14_1_slot → 18
UC14_1_teacher → rob
UC14_2_room → Lab01
UC14_2_slot → 10
UC14_2_teacher → rob
UC15_1_room → Lab01
UC15_1_slot → 4
UC15_1_teacher → sue
UC15_2_room → Room2
UC15_2_slot → 14
UC15_2_teacher → sue
UC21_1_room → Room1
UC21_1_slot → 1
UC21_1_teacher → jo
UC21_2_room → Online
UC21

### 9. Critical Analysis and Future Improvements

#### 9.1 Critical Analysis

The obtained results demonstrate that the constraint model is effective in producing valid and optimized timetables under the given constraints.
However, several observations and limitations were identified:

- **Solution diversity** - Although multiple valid schedules were found, many are similar, differing only in minor slot changes.

- **Execution time** - The CP-SAT solver performs well within 30 seconds, but the search space grows exponentially as more courses and teachers are added.

- **Soft constraint optimization** - The evaluation currently focuses only on minimizing room changes; future versions could include multiple weighted criteria (e.g., consecutive classes, teacher idle time).

- **Realism** - Some timetables, while valid, might still be impractical (e.g., too many gaps in a day).

#### 9.2 Future Improvements

To enhance the performance and quality of the agent, the following improvements are proposed:

1. **Multi-objective optimization** - Combine several soft constraints with different priorities (e.g., minimize room changes, maximize daily compactness).

2. **Adaptive heuristics** - Dynamically adjust heuristics during search (e.g., switch between MRV and randomization).

3. **Hybrid approach** - Combine constraint programming with local search (e.g., simulated annealing or genetic algorithms) for post-optimization.

4. **Visualization** - Create a timetable visualization (using matplotlib or a web UI) to make results easier to interpret.

5. **Scalability testing** - Evaluate performance on larger datasets with more teachers, courses, and rooms.

### 10. Conclusion

The development of this project allowed the team to design and implement an **intelligent agent** capable of automatically generating valid and optimized class timetables using **Constraint Programming (CP)** techniques.  

Throughout the project, several key insights were gained regarding **problem formulation, constraint modeling, and heuristic optimization**.  
The chosen approach — based on the **Google OR-Tools CP-SAT solver** — proved highly effective for managing complex scheduling requirements, offering both flexibility and computational efficiency.

The agent successfully handled multiple **hard constraints**, such as:
- Avoiding overlaps between classes, teachers, and rooms;  
- Enforcing teacher availability and room restrictions;  
- Respecting daily limits on the number of lessons per class;  
- Managing online sessions and lab allocations.  

Additionally, the implementation integrated **soft constraints** and a **custom evaluation function** to improve the overall quality of timetables by minimizing room changes and promoting balanced weekly schedules.

The development process followed an iterative methodology:
1. Initial formulation of the timetable as a **Constraint Satisfaction Problem (CSP)**.  
2. Incremental integration of new constraints and validation rules.  
3. Experimentation with different **heuristics** (MRV and LCV) to optimize solver performance.  
4. Implementation of a **custom solution collector** for post-processing and scoring.  
5. Export of the final best solution in a structured JSON format for visualization and analysis.  

The use of **Python** and **OR-Tools** proved to be a powerful combination for tackling CSP-based scheduling problems.  
Python provided readability and rapid prototyping capabilities, while OR-Tools offered a robust and high-performance constraint solver.  
These tools together enabled the creation of a flexible, extensible, and efficient solution.

From a broader perspective, the project demonstrated the **practical applicability of Artificial Intelligence techniques** to real-world problems, such as academic timetable generation — a task traditionally performed manually and prone to conflicts or inefficiencies.

Future work could explore:
- Incorporating **multi-objective optimization** to balance several soft constraints simultaneously;  
- Enhancing **solution visualization** through graphical interfaces;  
- Integrating **metaheuristic algorithms** (e.g., genetic algorithms, simulated annealing) for hybrid optimization;  
- Conducting scalability tests with larger datasets to assess solver performance.  

In conclusion, the agent met its objectives by generating valid and optimized timetables that adhere to institutional constraints and preferences.  
The project provided valuable experience in **AI problem modeling**, **constraint programming**, and **algorithmic reasoning**, reinforcing the importance of structured approaches in solving combinatorial optimization problems.
